# MiniFlow

In this lesson, I'll build a miniflow, a module that stores a simple neural network, implemented in python using numpy. The structure of this code and most of the implementations were created by instructors from Udacity's Deep Learning Foundation, while some implementation as created by me as exercises for the Nanodegree.


## MiniFlow Architecture

A Python class we'll ve used to represent a generic node. Each node will receive input from multiple other nodes, and also creates a single output that will likely be passed to other nodes.

In [1]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        # Node(s) from which this node receives values
        self.inbound_nodes = inbound_nodes
        
        # Node(s) to which this node passes values
        self.outbound_nodes = []
        
        # For each inbound nodes here, add this node as an inbound node to that node
        for node in self.inbound_nodes:
            node.outbound_nodes.append(self)
            
        # Initializing the value that will be passed to other nodes as None
        self.value = None
         
    def forward(self):
        """
        Forward propagation.
        
        Compute the output value based on inbound_nodes and
        store the result in self.value. It doesn't actually perform the forward pass,
        only calculate its value and stores it in self.value
        """
        raise NotImplemented

The class node only set the base set of properties every node holds, but only specialized subclasses of Node will end up in the graph.

In [2]:
class Input(Node):
    def __init__(self):
        # Since the input is the first node in the graph, it has no inbound_nodes
        # so, when initializing the Node class, there's no need to pass in any other nodes
        Node.__init__(self)
        
    def forward(self, value=None):
        """
        This is the only node where the value may be passe din as an argument for
        forward method, since it does not have to perform and operation with values from inbound nodes
        """
        # Remember that self.value was already initiated in Node.__init__(self)
        # Overwrite the value if one is passed in
        if value is not None:
            self.value = value
        

## The Add Subclass

The Add subclass actually performs a calculation, addition.

In [3]:
class Add(Node):
    """
    This class will take a list of nodes and add the values stored in them together
    """
    def __init__(self, *):
        """
        We'll initialize the Node (parent) init function with the arguments given 
        in to the Add class initialization. We'll pass them as a list, since the parent
        node has a list as an argument for inbound_nodes
        """
        Node.__init__(seld, list(inputs))
        
    def forward(self):
        """
        This method will add the values stored in inbound_nodes
        """
        summ = 0
        for node in self.inbound_nodes:
            summ += node.value
        self.value = summ
        

SyntaxError: named arguments must follow bare * (<ipython-input-3-67ee5fb841f4>, line 5)

Since the input of some node depends on the output of other, there are dependencies for the order of the operations. To arrange the nodes in a order such that the operations can be performed, we'll have to sort the nodes before applying the forward pass.
The topological_sort() function implements topological sorting using Kahn's Algorithm. This function returns a sorted list of nodes in which all of the calculations can run in series.

In [4]:
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.
    
    Arguments:
    'output_node': The output of the graph (no outgoing edges).
    'sorted_nodes': a topologically sorted list of nodes.
    
    Returns the output node's value
    """
    for n in sorted_nodes:
        n.forward()
        
    return output_node.value



Below will be defined the topological_sort() function, which implements topological sorting using Kahn's Algorithm.

In [5]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

## Forward Propagation

We'll now use the structures created above to perform a forward pass in our network

In [6]:
x, y, z = Input(), Input(), Input()

f = Add(x, y, z)

feed_dict = {x:4, y: 5, z:10}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

print("{} + {} + {} = {} (according to miniflow)".format(x.value, y.value, z.value, output))

NameError: name 'Add' is not defined

## Learning and Loss

So far, this neural network can only perform a forward pass through its nodes. The final objective is to improve the accuracy of their outputs over time, which is not useful for an Add node. So before we dive into the backpropagation part, a more complex node shall be implemented, the Linear node, which will perform a linear combination of a input nodes list, weights list and a bias.

In [7]:
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])
        # Since Linear node intantiate a regular Node with a list of lists, the
        # self.inbound_nodes isn't a list of nodes, but a list of three things:
        # list o inbound nodes, list of weights, and a bias
        
    def forward(self):
        linear_combination = 0
        
        for i in range(len(self.inbound_nodes[0].value)):
            linear_combination += self.inbound_nodes[0].value[i] * self.inbound_nodes[1].value[i]
        linear_combination += self.inbound_nodes[2].value
        
        self.value = linear_combination 
        
        """
        Using iteration with python structure is computationally poor, it's worth to mention
        that those python lists could have been transformed to python arrays and the dot product
        of them been performed to obtain the linear combination
        """

In [8]:
inputs, weights, bias = Input(), Input(), Input()

f = Linear(inputs, weights, bias)

feed_dict = {inputs:[6, 14, 3],
            weights:[0.5, 0.25, 1.4],
            bias: 2}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

print(output)

12.7


The previous example was performed with only one data point as input. Usually, it's common to feed in multiple data points in each forward pass, because the linear combinations can be processed in parallel, resulting in performance gains. The number of data points (exemples) is called batch size, and common numbers for batch size are 32, 64, 128, 256, 512.

So now the previous Linear node will be addapted to perform linear a linear transformation of the input matrix, which will be of size m (number of data points) by n (number of features of each exemple).

In [9]:
import numpy as np

class Linear(Node):
    def __init__(self, X, W, b):
        Node.__init__(self, [X, W, b])
        
    def forward(self):
        """
        This method will now perform a matrix multiplication between the features matrix
        and the weights matrix, and later add the bias array
        """
        inputs = self.inbound_nodes[0].value # m x n numpy matrix
        weights = self.inbound_nodes[1].value # n x k numpy matrix
        bias = self.inbound_nodes[2].value # vector of size k
        
        linear_transform = np.dot(inputs, weights)
        linear_transform += bias
        
        self.value = linear_transform
        
        
        

In [10]:
X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W:W_, b:b_}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

print(output)

[[-9.  4.]
 [-9.  4.]]


"Linear transforms are great for simply shifting values, but neural networks often require a more nuanced transform. For instance, one of the original designs for an artificial neuron, the perceptron, exhibit binary output behavior. Perceptrons compare a weighted input to a threshold. When the weighted input exceeds the threshold, the perceptron is activated and outputs 1, otherwise it outputs 0.

Activation, the idea of binary output behavior, generally makes sense for classification problems. For example, if you ask the network to hypothesize if a handwritten image is a '9', you're effectively asking for a binary output - yes, this is a '9', or no, this is not a '9'. A step function is the starkest form of a binary output, which is great, but step functions are not continuous and not differentiable, which is very bad. Differentiation is what makes gradient descent possible." 

## Sigmoid Function

As quoted above from the lesson of Deep Learning foundation course on MiniFlow, we need to use a differentiable funciton for the activation, so we can find a way to update the weights, in this case, using the gradient of the funtion. 

A sigmoid node must be created, one that takes as input a Linear node, perform and stores in its value the sigmoid of the input's value.

In [11]:
class Sigmoid(Node):
    """
    You need to fix the `_sigmoid` and `forward` methods.
    """
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used later with `backward` as well.

        `x`: A numpy array-like object.

        Return the result of the sigmoid function.

        Your code here!
        """
        return (1/(1+np.exp(-x)))
        
        


    def forward(self):
        """
        Set the value of this node to the result of the
        sigmoid function, `_sigmoid`.

        Your code here!
        """
        # This is a dummy value to prevent numpy errors
        # if you test without changing this method.
        
        
        self.value = self._sigmoid(self.inbound_nodes[0].value)

## Cost

There are still some structures needed to make a neural network learn upon the data it is given in. To measure how well the prediction of the network is, we'll use the Mean Square Error function, which evaluates how far your predicionts are form the correct labeled data from your training set.

In [12]:
class MSE(Node):
    def __init__(self, y, a):
        Node.__init__(self, [y, a])
        
    def forward(self):
        """
        since the matrices might not be of the same shape, it is needed to reshape them so they
        can be broadcasted together with no error"""
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        error = y-a
        squared_sum = np.square(error).sum()
        self.value = squared_sum/y.shape[0]
        
def forward_pass(graph):
    for n in graph:
        n.forward()

In [13]:
y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
forward_pass(graph)

"""
Expected output

23.4166666667
"""
print(cost.value)

23.4166666667


## Backpropagation

Ok, by now we have defined the types of nodes the neural network will have, Input, Linear, Sigmoid and MSE. Now we need to proceed to the most important part, the gradient descent step, which will enable the network to learn through the training process. The math behind the gradient step will only quickly be shown here, since the objective of this notebook is only to construct a simple network, showing the structures needed to make a working pattern recognition algorithm.

To remind how the gradient is propagated backwards through the network, here's a grate image from Udacity's course, which clearly shows the path and the respective derivatives thourhg this path, from the cost to the first weight matrix.

<img src="w1-backprop-graph.png" width=500px>

Since the nodes defined above will stro information about their inbound and outbound nodes, we can also implement the gradient steps within their definition. Below, it'll be shown the process for the linear node.

In [14]:
# Initialize a partial for each of the inbound_nodes
self.gradients = {n: np.zeros_like(n.value) for n in inbound_nodes}

# Cycle through the outputs. The gradient will change depending
# on each output, so the gradients are summed over all outputs

for n in self.outbound_nodes:
    # for each outbound node, we'll retrieve the gradient of that node with respect to this node
    grad_cost = n.gradients[self]
    
    # now we need to calculate and store the gradients with respect to the inputs of this node
    
    self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
    # this is multipling the error term comming to the linear node by the weights of the linear combination
    # so you get the error term, or the cost gradient, for the previous input node
    
    self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
    # This will yield the udate for the weights, by multiplying the error term comming to the linear node by the inputs to that node
    
    self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)
    
    

NameError: name 'inbound_nodes' is not defined

Now we'll need to update the Node class so it'll have a backward method and the attribute self.gradients. Also, the forward_pass() function will be replaced by another function, forward_and_bakward().

In [15]:
class Node(object):
    """
    Base class for nodes in the network.

    Arguments:

        `inbound_nodes`: A list of nodes with edges into this node.
    """
    def __init__(self, inbound_nodes=[]):
        """
        Node's constructor (runs when the object is instantiated). Sets
        properties that all nodes need.
        """
        # A list of nodes with edges into this node.
        self.inbound_nodes = inbound_nodes
        # The eventual value of this node. Set by running
        # the forward() method.
        self.value = None
        # A list of nodes that this node outputs to.
        self.outbound_nodes = []
        # New property! Keys are the inputs to this node and
        # their values are the partials of this node with
        # respect to that input.
        self.gradients = {}
        # Sets this node as an outbound node for all of
        # this node's inputs.
        for node in inbound_nodes:
            node.outbound_nodes.append(self)

    def forward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `forward` method.
        """
        raise NotImplementedError

    def backward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `backward` method.
        """
        raise NotImplementedError

def forward_and_backward(graph):
    """
    Performs a forward pass and a backward pass through a list of sorted nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # see: https://docs.python.org/2.3/whatsnew/section-slices.html
    for n in graph[::-1]:
        n.backward()

## Stochastic Gradient Descent

When training the network on a testing set, it would be ideal to pass all the data through the network simultaneously (in parallel), but since there's a memory limitation, we need to split the tesint set into batches and on every epoch, iterate over the batches. In this network, batches will be sampled randomly from the testing set.

After getting the sample from the data, we need to perform the gradient descent step, which will be implemented in the sgd_udate() function.

In [16]:
def sgd_update(trainables, learning_rate=1e-2):
    """
    Updates the values stored in the trainables nodes with SGD
    """
    
    for t in trainables:
        t.value -= learning_rate*t.gradients[t]
                

## Final Version of MiniFlow

Due to all changes made throughout the lesson, the MiniFlow will be rewritten as a whole, as a final version below, with all the node classes updated with their respective backward methods.

In [22]:
import numpy as np

class Node:
    """
    Base class for nodes in the network.

    Arguments:

        `inbound_nodes`: A list of nodes with edges into this node.
    """
    def __init__(self, inbound_nodes=[]):
        """
        Node's constructor (runs when the object is instantiated). Sets
        properties that all nodes need.
        """
        # A list of nodes with edges into this node.
        self.inbound_nodes = inbound_nodes
        # The eventual value of this node. Set by running
        # the forward() method.
        self.value = None
        # A list of nodes that this node outputs to.
        self.outbound_nodes = []
        # New property! Keys are the inputs to this node and
        # their values are the partials of this node with
        # respect to that input.
        self.gradients = {}
        # Sets this node as an outbound node for all of
        # this node's inputs.
        for node in inbound_nodes:
            node.outbound_nodes.append(self)

    def forward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `forward` method.
        """
        raise NotImplementedError

    def backward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `backward` method.
        """
        raise NotImplementedError


class Input(Node):
    """
    A generic input into the network.
    """
    def __init__(self):
        # The base class constructor has to run to set all
        # the properties here.
        #
        # The most important property on an Input is value.
        # self.value is set during `topological_sort` later.
        Node.__init__(self)

    def forward(self):
        # Do nothing because nothing is calculated.
        pass

    def backward(self):
        # An Input node has no inputs so the gradient (derivative)
        # is zero.
        # The key, `self`, is reference to this object.
        self.gradients = {self: 0}
        # Weights and bias may be inputs, so you need to sum
        # the gradient from output gradients.
        for n in self.outbound_nodes:
            self.gradients[self] += n.gradients[self]

class Linear(Node):
    """
    Represents a node that performs a linear transform.
    """
    def __init__(self, X, W, b):
        # The base class (Node) constructor. Weights and bias
        # are treated like inbound nodes.
        Node.__init__(self, [X, W, b])

    def forward(self):
        """
        Performs the math behind a linear transform.
        """
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b

    def backward(self):
        """
        Calculates the gradient based on the output values.
        """
        # Initialize a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            # Set the partial of the loss with respect to this node's inputs.
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            # Set the partial of the loss with respect to this node's weights.
            self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            # Set the partial of the loss with respect to this node's bias.
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)


class Sigmoid(Node):
    """
    Represents a node that performs the sigmoid activation function.
    """
    def __init__(self, node):
        # The base class constructor.
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used with `backward` as well.

        `x`: A numpy array-like object.
        """
        return 1. / (1. + np.exp(-x))

    def forward(self):
        """
        Perform the sigmoid function and set the value.
        """
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)

    def backward(self):
        """
        Calculates the gradient using the derivative of
        the sigmoid function.
        """
        # Initialize the gradients to 0.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Sum the partial with respect to the input over all the outputs.
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            sigmoid = self.value
            self.gradients[self.inbound_nodes[0]] += sigmoid * (1 - sigmoid) * grad_cost


class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)

        self.m = self.inbound_nodes[0].value.shape[0]
        # Save the computed output for backward.
        self.diff = y - a
        self.value = np.mean(self.diff**2)

    def backward(self):
        """
        Calculates the gradient of the cost.
        """
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff


def topological_sort(feed_dict):
    """
    Sort the nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Node and the value is the respective value feed to that Node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_and_backward(graph):
    """
    Performs a forward pass and a backward pass through a list of sorted Nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # see: https://docs.python.org/2.3/whatsnew/section-slices.html
    for n in graph[::-1]:
        n.backward()


def sgd_update(trainables, learning_rate=1e-1):
    """
    Updates the value of each trainable with SGD.

    Arguments:

        `trainables`: A list of `Input` Nodes representing weights/biases.
        `learning_rate`: The learning rate.
    """
    # Performs SGD
    #
    # Loop over the trainables
    for t in trainables:
        # Change the trainable's value by subtracting the learning rate
        # multiplied by the partial of the cost with respect to this
        # trainable.
        partial = t.gradients[t]
        t.value -= learning_rate * partial


In [23]:
"""
Check out the new network architecture and dataset!

Notice that the weights and biases are
generated randomly.

No need to change anything, but feel free to tweak
to test your network, play around with the epochs, batch size, etc!
"""

import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample


# Load data
data = load_boston()
X_ = data['data']
y_ = data['target']

# Normalize data
X_ = (X_ - np.mean(X_, axis=0)) / np.std(X_, axis=0)

n_features = X_.shape[1]
n_hidden = 10
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)
W2_ = np.random.randn(n_hidden, 1)
b2_ = np.zeros(1)

# Neural network
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(y, l2)

feed_dict = {
    X: X_,
    y: y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

epochs = 10
# Total number of examples
m = X_.shape[0]
batch_size = 11
steps_per_epoch = m // batch_size

graph = topological_sort(feed_dict)
trainables = [W1, b1, W2, b2]

print("Total number of examples = {}".format(m))

# Step 4
for i in range(epochs):
    loss = 0
    for j in range(steps_per_epoch):
        # Step 1
        # Randomly sample a batch of examples
        X_batch, y_batch = resample(X_, y_, n_samples=batch_size)

        # Reset value of X and y Inputs
        X.value = X_batch
        y.value = y_batch

        # Step 2
        forward_and_backward(graph)

        # Step 3
        sgd_update(trainables)

        loss += graph[-1].value

    print("Epoch: {}, Loss: {:.3f}".format(i+1, loss/steps_per_epoch))


Total number of examples = 506
Epoch: 1, Loss: 42.169
Epoch: 2, Loss: 35.741
Epoch: 3, Loss: 24.055
Epoch: 4, Loss: 18.483
Epoch: 5, Loss: 14.668
Epoch: 6, Loss: 16.385
Epoch: 7, Loss: 16.844
Epoch: 8, Loss: 14.931
Epoch: 9, Loss: 15.788
Epoch: 10, Loss: 11.587
